<a href="https://colab.research.google.com/github/HuynhLeVu5598/MQTT/blob/master/Detect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/paul-pias/Object-Detection-and-Distance-Measurement

https://blogs.nvidia.com/blog/2019/06/19/drive-labs-distance-to-object-detection/

https://www.diva-portal.org/smash/get/diva2:1479957/FULLTEXT01.pdf

https://github.com/E-Renshaw/object-detection-distance-ranging

https://github.com/Derekabc/In-vehicle-Driving-Quality-Monitoring

https://morioh.com/p/e5e1eebb89b6

https://scholar.uwindsor.ca/cgi/viewcontent.cgi?article=9147&context=etd

In [ ]:
%cd drive/MyDrive/mabuchi/

/content/drive/MyDrive/mabuchi


In [ ]:
!git clone https://github.com/Asadullah-Dal17/Distance_measurement_using_single_camera.git

Cloning into 'Distance_measurement_using_single_camera'...
remote: Enumerating objects: 278, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 278 (delta 36), reused 66 (delta 18), pack-reused 187
Receiving objects: 100% (278/278), 187.68 MiB | 20.12 MiB/s, done.
Resolving deltas: 100% (103/103), done.
Checking out files: 100% (70/70), done.


In [ ]:
%cd Distance_measurement_using_single_camera/

/content/drive/My Drive/mabuchi/Distance_measurement_using_single_camera


In [ ]:
%cd Capture_Reference_image/

/content/drive/My Drive/mabuchi/Distance_measurement_using_single_camera/Capture_Reference_image


In [ ]:
# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time

In [ ]:
# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

In [ ]:
# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'
# initialze bounding box to empty
#bbox = ''
#count = 0 
while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    ref_img = js_to_image(js_reply["img"])

    # convert JS response to OpenCV Image
    #img = js_to_image(js_reply["img"])


<IPython.core.display.Javascript object>

In [ ]:
def FocalLength(measured_distance, real_width, width_in_rf_image):
    '''
This Function Calculate the Focal Length(distance between lens to CMOS sensor), it is simple constant we can find by using 
MEASURED_DISTACE, REAL_WIDTH(Actual width of object) and WIDTH_OF_OBJECT_IN_IMAGE 
:param1 Measure_Distance(int): It is distance measured from object to the Camera while Capturing Reference image

:param2 Real_Width(int): It is Actual width of object, in real world (like My face width is = 14.3 centimeters)
:param3 Width_In_Image(int): It is object width in the frame /image in our case in the reference image(found by Face detector) 
:retrun Focal_Length(Float):
'''
    focal_length = (width_in_rf_image * measured_distance) / real_width
    return focal_length
# distance estimation function


def Distance_finder(Focal_Length, real_face_width, face_width_in_frame):
    '''
This Function simply Estimates the distance between object and camera using arguments(Focal_Length, Actual_object_width, Object_width_in_the_image)
:param1 Focal_length(float): return by the Focal_Length_Finder function

:param2 Real_Width(int): It is Actual width of object, in real world (like My face width is = 5.7 Inches)
:param3 object_Width_Frame(int): width of object in the image(frame in our case, using Video feed)
:return Distance(float) : distance Estimated  

'''
    distance = (real_face_width * Focal_Length)/face_width_in_frame
    return distance


def face_data(image):
    '''
    This function Detect the face 
    :param Takes image as argument.
    :returns face_width in the pixels
    '''

    face_width = 0
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(gray_image, 1.3, 5)
    for (x, y, h, w) in faces:
        cv2.rectangle(image, (x, y), (x+w, y+h), WHITE, 1)
        face_width = w

    return face_width

# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes



In [ ]:
import cv2

video_stream()
# label for video
label_html = 'Capturing...'
bbox = ''

# variables
# distance from camera to object(face) measured
Known_distance = 76.2  # centimeter
# width of face in the real world or Object Plane
Known_width = 14.3  # centimeter
# Colors
GREEN = (0, 255, 0)
RED = (0, 0, 255)
WHITE = (255, 255, 255)
fonts = cv2.FONT_HERSHEY_COMPLEX
#cap = cv2.VideoCapture(1)
# face detector object
#face_detector = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
face_detector = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))
# focal length finder function



while True:
    frame = video_frame(label_html, bbox)
    if not frame:
        break

    
    img = js_to_image(frame["img"])
    
    bbox_array = np.zeros([480,640,4], dtype=np.uint8)
    #ref_image = cv2.imread("Ref_image.png")

    #ref_image_face_width = face_data(ref_image)
    ref_image_face_width = 0
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    faces = face_detector.detectMultiScale(gray_image)
    for (x, y, h, w) in faces:
        bbox_array = cv2.rectangle(image, (x, y), (x+w, y+h), WHITE, 1)
        ref_image_face_width = w



    Focal_length_found = FocalLength(Known_distance, Known_width, ref_image_face_width)
    print(Focal_length_found)

    bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
    # convert overlay of bbox into bytes
    bbox_bytes = bbox_to_bytes(bbox_array)
    #print(bbox_bytes)
    # update bbox so next frame gets new overlay
    bbox = bbox_bytes

    cv2.imshow("ref_image", ref_image)
    # calling face_data function
    face_width_in_frame = face_data(frame)
    # finding the distance by calling function Distance
    if face_width_in_frame != 0:
        Distance = Distance_finder(
            Focal_length_found, Known_width, face_width_in_frame)
    # Drwaing Text on the screen




<IPython.core.display.Javascript object>

0.0


DisabledFunctionError: ignored